In [27]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [28]:
#Import variables with static values and store in dictionary
static_values_df = pd.read_excel("static_values.xlsx")
static_values_dict_df = static_values_df[["Variable_name","Value"]]
static_values_dict_df = static_values_dict_df.set_index(['Variable_name'])
static_values_dict = static_values_dict_df.to_dict()
static_values_dict = static_values_dict['Value']
static = static_values_dict

In [29]:
#import daily inputs
daily_inputs = pd.read_excel('daily_inputs.xlsx')

In [30]:
#create empty output df
column_names = ['Date','New_COVID_Cases','Runs_with_PPE', 'EMS_max_precaution', 'EMS_moderate_precaution', 'EMS_staff_working','gloves_used','gowns_used', 'N95s_used','eye_pro_used','surg_proc_masks_used']
outputs = pd.DataFrame(columns = column_names)

In [31]:
#determine PPE use for each day in daily inputs and updates outputs df

day = 0
Max_Precaution = False
Moderate_Precaution = False

for i in range(len(daily_inputs)):
    
    #updates EMS precaution level
    if daily_inputs.at[day,'New_COVID_Cases'] >= static['COVID_threshold_for_full_PPE_use']:
        Full_Precaution = True
        High_Precaution = False
        Moderate_Precaution = False
    elif daily_inputs.at[day,'New_COVID_Cases'] >= static['COVID_threshold_for_high_PPE_use']:
        Full_Precaution = False
        High_Precaution = True
        Moderate_Precaution = False
    else:
        Full_Precaution = False
        High_Precaution = False
        Moderate_Precaution = True
        
    #calculates EMS Staff on runs
    EMS_staff = round((static['Daily_runs'] / static['Runs_per_EMS_shift']) * static['Staff_per_EMS_run'] )
    
    
    #calculates EMS runs with full COVID PPE
    if Full_Precaution == True:
        Runs_with_PPE = static['Daily_runs']
    elif High_Precaution == True:
        Runs_with_PPE = static['Daily_runs'] * (((.6/9000)* daily_inputs.at[day,'New_COVID_Cases']) + .4)
    elif Moderate_Precaution == True:
        Runs_with_PPE = static['Daily_runs'] * (.0004 * daily_inputs.at[day,'New_COVID_Cases'])
    
    #calculates PPE usage
    
    N95s = round((EMS_staff) / static['Shifts_per_N95'])
    Eye_protection = round(EMS_staff/ static['Shifts_per_eyepro'])
    Surgical_Procedural_mask = ((1 - (Runs_with_PPE / static['Daily_runs'])) * static['Daily_runs'] * static['Staff_per_EMS_run']) + static['Daily_runs']  
    Gowns = Runs_with_PPE * static['Staff_per_EMS_run']
    Gloves = static['Daily_runs'] * static['Staff_per_EMS_run']*2
    
    #updates the outputs
    outputs = outputs.append({'Runs_with_PPE':Runs_with_PPE,'Date': daily_inputs.at[day, 'Date'],'New_COVID_Cases': daily_inputs.at[day, 'New_COVID_Cases'], 'EMS_max_precaution': Max_Precaution, 'EMS_moderate_precaution': Moderate_Precaution, 'EMS_staff_working': EMS_staff, 'gloves_used': Gloves,'gowns_used': Gowns, 'N95s_used': N95s,'eye_pro_used': Eye_protection,'surg_proc_masks_used': Surgical_Procedural_mask}, ignore_index = True)
    
    day += 1                                                                                      

In [14]:
#export outputs to excel
outputs.to_excel('PPE_outputs.xlsx')